In [ ]:
# http://127.0.0.1:8888/?token=d8645dfa36d67d3f2ca03610ce0c3c8e01cc36bd6dfab090
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
sns.set(style="whitegrid", color_codes=True)
sns.set(font_scale=1.4)
color = sns.color_palette()
from sklearn.decomposition import PCA

## 读取数据以及数据预览
***

In [ ]:
train_data = pd.read_csv('C:/Users/54wy/Desktop/AI Challenger/Stock/Stock_Week10/inputs/stock_train_data_20171103.csv')
test_data = pd.read_csv('C:/Users/54wy/Desktop/AI Challenger/Stock/Stock_Week10/inputs/stock_test_data_20171103.csv')
print('Shape train_data: {}\nShape test_data: {}'.format(train_data.shape,test_data.shape))

In [ ]:
# train_data.head()

In [ ]:
# test_data.head()

In [ ]:
train_data.describe().T

In [ ]:
test_data.describe().T

In [ ]:
train_data.isnull().sum()

## 数据预处理 
***

* 去掉无用列

In [ ]:
X_train = train_data.drop(['id','weight','era','group1','group2','code_id'], axis=1)
y_train = X_train.pop('label')
X_train_group1 = train_data['group1']
X_train_group2 = train_data['group2']
X_train_code_id = train_data['code_id']
X_test = test_data.drop(['id','group1','group2','code_id'], axis=1)
X_test_group1 = test_data['group1']
X_test_group2 = test_data['group2']
X_test_code_id = test_data['code_id']

* One hot encoding 

In [ ]:
# group_train_dummies = pd.get_dummies(X_train['group'],prefix='group')
# group_test_dummies = pd.get_dummies(X_test['group'],prefix='group')

In [ ]:
# X_train.drop(['group'],axis=1,inplace=True)
# X_test.drop(['group'],axis=1,inplace=True)
# X_train=X_train.join(group_train_dummies)
# X_test=X_test.join(group_test_dummies)

## label列和group列以及weight列统计
***

In [ ]:
train_data['label'].value_counts()

In [ ]:
print('label 1 ratio: {}\n'.format(sum(train_data['label']) / train_data.shape[0]))
print('label 0 ratio: {}\n'.format(len(train_data[train_data['label'] == 0]) / train_data.shape[0]))

In [ ]:
sns.set(font_scale=1.8)
int_level = train_data['label'].value_counts()
plt.figure(figsize=(20,5))
sns.barplot(int_level.index, int_level.values, alpha=0.8, color='b')
plt.title('Counts of label in train_data')
plt.show()

In [ ]:
train_data['group1'].value_counts()

In [ ]:
test_data['group1'].value_counts()

In [ ]:
sns.set(font_scale=1.8)
int_level = train_data['group1'].value_counts()
plt.figure(figsize=(20,5))
sns.barplot(int_level.index, int_level.values, alpha=0.8, color='b')
plt.title('Counts of group1 in train_data')
plt.show()

In [ ]:
sns.set(font_scale=1.8)
int_level = test_data['group1'].value_counts()
plt.figure(figsize=(20,5))
sns.barplot(int_level.index, int_level.values, alpha=0.8, color='r')
plt.title('Counts of group1 in test_data')
plt.show()

In [ ]:
train_data['group2'].value_counts()

In [ ]:
test_data['group2'].value_counts()

In [ ]:
sns.set(font_scale=0.8)
int_level = train_data['group2'].value_counts()
plt.figure(figsize=(20,8))
sns.barplot(int_level.index, int_level.values, alpha=0.8, color='b')
plt.title('Counts of group2 in train_data')
plt.show()

In [ ]:
sns.set(font_scale=0.8)
int_level = test_data['group2'].value_counts()
plt.figure(figsize=(20,8))
sns.barplot(int_level.index, int_level.values, alpha=0.8, color='r')
plt.title('Counts of group2 in test_data')
plt.show()

In [ ]:
train_data['code_id'].value_counts()

In [ ]:
test_data['code_id'].value_counts()

## Feature Correlation 以及 Importance
***

In [ ]:
train_data = train_data.drop(['id','weight','era'], axis=1)

In [ ]:
corrmat = train_data.corr()
#f, ax = plt.subplots(figsize=(500, 500))
#sns.heatmap(corrmat, vmax=.8, square=True);

In [ ]:
k = 15 #number of variables for heatmap
plt.figure(figsize=(20, 15))
cols = corrmat.nlargest(k, 'label')['label'].index
cm = np.corrcoef(train_data[cols].values.T)
sns.set(font_scale=1.75)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, 
fmt='.4f', annot_kws={'size': 15}, yticklabels=cols.values, 
xticklabels=cols.values)
plt.show()

In [ ]:
numeric_features = train_data.select_dtypes(include=[np.number])
numeric_features.dtypes
corr = numeric_features.corr()
print("%d features in total" %(len(corr)))
print (corr['label'].sort_values(ascending=False)[:90], '\n')
#print (corr['label'].sort_values(ascending=False)[-5:])

## 查看Feature分布
***

In [ ]:
sns.set(font_scale=1.8)
plt.figure(figsize=(20, 10))
sns.distplot(X_train['feature0'], bins=100 ,kde_kws={'lw': 2.5, 'color': 'k', 'alpha': 0.7, 'label': 'KDE'}, hist_kws={ 'histtype': 'step', 'lw': 3, 'color': 'b', 'alpha': 0.9, 'label': 'feature0'})
plt.title('Distribution of Feature0 in X_train')
plt.show()

In [ ]:
sns.set(font_scale=1.8)
plt.figure(figsize=(20, 10))
sns.distplot(X_test['feature0'], bins=100 ,kde_kws={'lw': 2.5, 'color': 'k', 'alpha': 0.7, 'label': 'KDE'}, hist_kws={ 'histtype': 'step', 'lw': 3, 'color': 'r', 'alpha': 0.9, 'label': 'feature0'})
plt.title('Distribution of Feature0 in X_test')
plt.show()

In [ ]:
sns.set(font_scale=1.8)
plt.figure(figsize=(20, 10))
ax1 = sns.distplot(X_train['feature0'], bins=100, kde=True, color='b', kde_kws={'lw': 3, 'label':'X_train'}, hist=False)
ax2 = sns.distplot(X_test['feature0'], bins=100, kde=True, color='r', kde_kws={'lw': 3, 'label':'X_test'}, hist=False)
plt.title('Distribution of Feature0 in X_train and X_test');
ax1.set_xlim([-2,5])
ax2.set_xlim([-2,5])
plt.show()

In [ ]:
sns.set(font_scale=1.8)
plt.figure(figsize=(20, 10))
plt.scatter(range(X_train.shape[0]), train_data["feature0"].values, color='m')
plt.title('Sactter distribution of Feature0 in X_train')
plt.show()

In [ ]:
sns.set(font_scale=1.8)
plt.figure(figsize=(20, 10))
plt.scatter(range(X_test.shape[0]), test_data["feature0"].values, color='g')
plt.title('Sactter distribution of Feature0 in X_test')
plt.show()

* 训练集和测试集feature0-104以及group分布对比

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2, figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[0], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[0], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[1], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[1], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[2], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[2], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[3], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[3], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[4], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[4], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[5], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[5], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[6], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[6], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[7], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[7], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[8], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[8], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[9], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[9], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[10], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[10], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[11], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[11], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[12], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[12], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[13], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[13], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[14], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[14], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[15], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[15], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[16], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[16], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[17], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[17], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[18], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[18], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[19], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[19], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[20], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[20], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[21], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[21], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[22], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[22], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[23], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[23], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[24], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[24], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[25], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[25], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[26], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[26], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[27], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[27], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[28], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[28], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[29], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[29], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[30], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[30], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[31], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[31], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[32], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[32], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[33], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[33], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[34], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[34], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[35], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[35], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[36], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[36], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[37], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[37], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[38], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[38], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[39], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[39], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[40], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[40], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[41], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[41], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[42], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[42], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[43], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[43], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[44], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[44], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[45], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[45], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[46], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[46], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[47], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[47], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[48], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[48], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[49], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[49], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[50], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[50], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[51], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[51], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[52], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[52], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[53], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[53], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[54], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[54], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[55], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[55], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[56], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[56], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[57], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[57], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[58], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[58], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[59], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[59], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[60], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[60], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[61], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[61], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[62], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[62], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[63], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[63], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[64], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[64], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[65], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[65], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[66], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[66], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[67], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[67], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[68], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[68], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[69], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[69], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[70], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[70], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[71], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[71], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[72], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[72], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[73], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[73], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[74], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[74], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[75], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[75], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[76], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[76], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[77], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[77], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[78], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[78], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[79], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[79], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[80], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[80], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[81], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[81], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[82], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[82], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[83], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[83], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[84], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[84], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[85], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[85], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[86], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[86], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[87], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[87], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[88], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[88], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[89], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[89], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[90], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[90], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[91], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[91], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[92], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[92], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[93], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[93], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[94], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[94], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[95], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[95], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[96], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[96], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[97], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[97], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[98], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[98], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[99], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[99], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[100], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[100], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[101], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[101], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[102], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[102], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[103], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[103], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=X_train.columns[104], data=X_train, ax=axes[0])
cx = sns.violinplot(y=X_test.columns[104], data=X_test, ax=axes[1])
bx.set_ylim([-3,3])
cx.set_ylim([-3,3])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=train_data['group1'], data=X_train, ax=axes[0])
cx = sns.violinplot(y=test_data['group1'], data=X_test, ax=axes[1])
bx.set_ylim([-1,28])
cx.set_ylim([-1,28])
plt.show()

In [ ]:
sns.set(font_scale=1.4)
fg,axes = plt.subplots(1,2,figsize=(20, 5),sharex=True)
bx = sns.violinplot(y=train_data['group2'], data=X_train, ax=axes[0])
cx = sns.violinplot(y=test_data['group2'], data=X_test, ax=axes[1])
bx.set_ylim([-1,104])
cx.set_ylim([-1,104])
plt.show()

#train_data.info()

In [ ]:
#train_data.isnull().sum()

In [ ]:
#table_type = train_data.dtypes.reset_index()
#table_type.columns=['feat','type']
#print(table_type.groupby('type').aggregate('count'))

In [ ]:
#sns.countplot(train_data.label,order=['0', '1']);
#plt.xlabel('label');
#plt.ylabel('Counts');

# PCA处理

* 对[group]列28个虚拟变量做PCA处理

In [ ]:
pca = PCA(n_components=28)

In [ ]:
pca.fit(group_train_dummie)

In [ ]:
print(pca.explained_variance_ratio_)
print(pca.explained_variance_)
print(pca.n_components_)

* 对feature0-feature88做PCA处理

In [ ]:
pca = PCA(n_components=88)

In [ ]:
pca.fit_transform(X_train)

In [ ]:
print(pca.explained_variance_ratio_)
print(pca.explained_variance_)
print(pca.n_components_)

In [ ]:
pca = PCA(n_components=88)